## Enable Amazon SageMaker Model Monitor for XGBOOST - model quality

## Step 1: Setup

In [170]:

%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

CPU times: user 46.6 ms, sys: 7.78 ms, total: 54.4 ms
Wall time: 55.2 ms


In [171]:
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

RoleArn: arn:aws:iam::246778806733:role/service-role/AmazonSageMaker-ExecutionRole-20201111T111846
Region: us-east-1


In [173]:
#Setup S3 bucket
# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket =  session.default_bucket()
print("XGboost Model Quality:", bucket)
prefix = 'sagemaker/fraud-ModelQualityMonitor'

##S3 prefixes
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f's3://{bucket}/{data_capture_prefix}'

ground_truth_upload_path = f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"

reports_prefix = f'{prefix}/reports'
s3_report_path = f's3://{bucket}/{reports_prefix}'

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

XGboost Model Quality: sagemaker-us-east-1-246778806733
Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/datacapture
Ground truth path: s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/ground_truth_data/2021-01-14-13-48-22
Report path: s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/reports


## PART A: Capturing real-time inference data from Amazon SageMaker endpoints

### Upload the pre-trained model to Amazon S3

In [174]:
import boto3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefixmodel, 'model.tar.gz')).download_file('model.tar.gz')

model_file = open("model.tar.gz", 'rb')
s3_key = os.path.join(prefix, 'xgb-fraud-prediction-model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

In [175]:
from time import gmtime, strftime
from sagemaker.model import Model
#from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.image_uris import retrieve

model_name = "xbg-bank-fraud-model-quality-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://{}.s3-{}.amazonaws.com/{}/xgb-fraud-prediction-model.tar.gz'.format(bucket, region, prefix)
#image_uri = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-2')
container = retrieve(framework='xgboost',region=boto3.Session().region_name, version='1.2-1')



model = Model(image_uri=container, model_data=model_url, role=role)

In [176]:
model_name

'xbg-bank-fraud-model-quality-monitor-2021-01-14-13-51-49'

In [177]:
model_url

'https://sagemaker-us-east-1-246778806733.s3-us-east-1.amazonaws.com/sagemaker/fraud-ModelQualityMonitor/xgb-fraud-prediction-model.tar.gz'

In [178]:
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = 'xbg-bank-fraud-monitor-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=50,
                        destination_s3_uri=s3_capture_upload_path)

predictor = model.deploy(initial_instance_count=1,
                instance_type='ml.m5.xlarge',
                endpoint_name=endpoint_name,
                data_capture_config=data_capture_config)

EndpointName=xbg-bank-fraud-monitor-2021-01-14-13-52-13
-------------!

In [211]:
###create predictor#from sagemaker.predictor import RealTimePredictor
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
import time

#predictor = RealTimePredictor(endpoint=endpoint_name,content_type='text/csv')
predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=session, serializer=CSVSerializer())



### Invoke the deployed model

In [215]:

cutoff = 0.50
#validate_dataset = "valdiation_with_predictions.csv"

In [216]:

# get a subset of test data for a quick test

limit = 2000 #Need at least 200 samples to compute standard deviations
i = 0
with open("valdiation_with_predictions.csv", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n") # our header
    with open('test_data_tail_with_label.csv', 'r') as f:
        for row in f:
            print(".", sep=',', end='', flush=True)
            (label, input_cols) = row.split(",", 1)
            input_cols = input_cols.rstrip()
            probability = float(predictor.predict(input_cols))
            prediction = "1" if probability > cutoff else "0"
            baseline_file.write(f"{probability},{prediction},{label}\n")
            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            sleep(0.5)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [218]:
!head valdiation_with_predictions.csv

probability,prediction,label
1.1669402510960936e-06,0,0
2.5927639057954366e-07,0,0
2.8943654797330964e-06,0,0
2.072722224966128e-07,0,0
1.965755018318305e-06,0,0
3.0075946142460452e-06,0,0
1.0677201771613909e-06,0,0
3.6262547382648336e-06,0,0
6.11385758020333e-07,0,0


In [219]:
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = f's3://{bucket}/{baseline_data_prefix}'
baseline_results_uri = f's3://{bucket}/{baseline_results_prefix}'
print(f'Baseline data uri: {baseline_data_uri}')
print(f'Baseline results uri: {baseline_results_uri}')

Baseline data uri: s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/baselining/results


In [221]:

baseline_dataset_uri = S3Uploader.upload("valdiation_with_predictions.csv", baseline_data_uri)
baseline_dataset_uri

's3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/baselining/data/valdiation_with_predictions.csv'

In [222]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

In [223]:
#Create the model quality monitoring object
fraud_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session
)

In [224]:
#Name of the model quality baseline job
baseline_job_name = f"xgb-fraud-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"

In [225]:
#Execute the baseline suggestion job. 
#You will specify problem type, in this case Binary Classification, and provide other required attributes.
job = fraud_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri = baseline_results_uri,
    problem_type='BinaryClassification',
    inference_attribute= "prediction",
    probability_attribute= "probability",
    ground_truth_attribute= "label"
)
job.wait(logs=False)


Job Name:  xgb-fraud-model-baseline-job-2021-01-14-1521
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/baselining/data/valdiation_with_predictions.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
................................................................!

In [226]:
baseline_job = fraud_model_quality_monitor.latest_baselining_job

In [227]:
binary_metrics = baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
pd.json_normalize(binary_metrics).T

,0
confusion_matrix.0.0,1997
confusion_matrix.0.1,0
confusion_matrix.1.0,0
confusion_matrix.1.1,3
recall.value,1
recall.standard_deviation,0
precision.value,1
precision.standard_deviation,0
accuracy.value,1
accuracy.standard_deviation,0


In [228]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,1,LessThanThreshold
precision,1,LessThanThreshold
accuracy,1,LessThanThreshold
true_positive_rate,1,LessThanThreshold
true_negative_rate,1,LessThanThreshold
false_positive_rate,0,GreaterThanThreshold
false_negative_rate,0,GreaterThanThreshold
auc,1,LessThanThreshold
f0_5,1,LessThanThreshold
f1,1,LessThanThreshold


## Setup continuous model monitoring to identify model quality drift

In [229]:
def invoke_endpoint(ep_name, file_name):    
    with open(file_name, 'r') as f:
        i = 0
        for row in f:
            payload = row.rstrip('\n')
            response = session.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType='text/csv', 
                Body=payload,
                InferenceId=str(i), # unique ID per row
            )["Body"].read()
            i += 1
            sleep(1)
            
def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, 'validation_data_tail.csv')
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

In [230]:
print("Waiting for captures to show up", end="")
for _ in range(120):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

Waiting for captures to show up...............................................
Found Capture Files:
s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/datacapture/xbg-bank-fraud-monitor-2021-01-14-13-52-13/AllTraffic/2021/01/14/15/02-07-879-6aec8986-0a70-4af3-8688-5f1b5118752d.jsonl
 s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/datacapture/xbg-bank-fraud-monitor-2021-01-14-13-52-13/AllTraffic/2021/01/14/15/03-11-147-6c42dfdd-6df4-4ff1-b8dd-293afd70ee1f.jsonl
 s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/datacapture/xbg-bank-fraud-monitor-2021-01-14-13-52-13/AllTraffic/2021/01/14/15/29-31-789-17bcd6b6-77f7-4281-b78e-7bc27f10096a.jsonl


In [231]:
print("\n".join(capture_file[-3:-1]))

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0.3800539083557952,0.0015271360378650808,0.05377653166134794,0.0674690121261668,0.00223402722685248,0.0018366377515939972,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.999999999891828,0.11643378368927686","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"7.814322202648327e-07","encoding":"CSV"}},"eventMetadata":{"eventId":"d16f682f-e06d-40a7-b141-8d403f838597","inferenceId":"59","inferenceTime":"2021-01-14T15:30:30Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0.43126684636118606,0.0026003318358435857,1.1747915175575472e-06,0.0,0.0005696368225875084,0.0012442856904641634,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.997400425259022,0.11643378368927686","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"4.3943131800006086e-07","encoding":"CSV"}},"eventM

In [232]:
print(json.dumps(capture_record, indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "0.3450134770889488,0.001907589317573205,0.0,0.0,0.000527946829404624,0.0010228150584858282,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.9980924105744612,0.11643378357284305",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "2.3778798095008824e-06",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "f9fbd1d4-50f2-4beb-891a-2c49286be7a1",
    "inferenceId": "1",
    "inferenceTime": "2021-01-14T15:29:31Z"
  },
  "eventVersion": "0"
}


In [233]:
###generate synthetic truth data
import random

def ground_truth_with_id(inference_id):
    random.seed(inference_id) # to get consistent results
    rand = random.random()
    return {
        'groundTruthData': {
            'data': "1" if rand < 0.3 else "0", # randomly generate positive labels 30% of the time
            'encoding': 'CSV'
        },
        'eventMetadata': {
            'eventId': str(inference_id),
        },
        'eventVersion': '0',
    }

def upload_ground_truth(records, upload_time):
    fake_records = [ json.dumps(r) for r in records ]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [234]:
NUM_GROUND_TRUTH_RECORDS = 2000 # 2000 are the number of rows in data we're sending for inference

def generate_fake_ground_truth_forever():
    j = 0
    while True:
        fake_records = [ ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS) ]
        upload_ground_truth(fake_records, datetime.utcnow())
        j = (j + 1) % 5
        sleep(60*60) # do this once an hour

gt_thread = Thread(target = generate_fake_ground_truth_forever)
gt_thread.start()

Uploading 2000 records to s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/ground_truth_data/2021-01-14-13-48-22/2021/01/14/15/3055.jsonl


## create the schedule

In [235]:
##Monitoring schedule name
fraud_monitor_schedule_name = f"xgb-fraud-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}"

In [236]:
fraud_monitor_schedule_name

'xgb-fraud-monitoring-schedule-2021-01-14-1531'

In [237]:
#Create an enpointInput 
endpointInput = EndpointInput(endpoint_name=predictor.endpoint_name, 
                              probability_attribute="0", 
                              probability_threshold_attribute=0.5,
                              destination='/opt/ml/processing/input_data')

In [239]:
#Create the monitoring schedule to execute every hour.
from sagemaker.model_monitor import CronExpressionGenerator

response = fraud_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=fraud_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri = baseline_results_uri,
    problem_type='BinaryClassification',
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(), 
    enable_cloudwatch_metrics=True
)

In [240]:
#Create the monitoring schedule
#You will see the monitoring schedule in the 'Scheduled' status
fraud_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:246778806733:monitoring-schedule/xgb-fraud-monitoring-schedule-2021-01-14-1531',
 'MonitoringScheduleName': 'xgb-fraud-monitoring-schedule-2021-01-14-1531',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2021, 1, 14, 15, 31, 59, 850000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 1, 14, 15, 32, 2, 159000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2021-01-14-15-31-59-579',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'xbg-bank-fraud-monitor-2021-01-14-13-52-13',
 'ResponseMetadata': {'RequestId': '4f9faef7-07fc-4e82-970a-fa3d649845e0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4f9faef7-07fc-4e82-970a-fa3d649845e0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '609',

## Examine monitoring schedule executions

In [242]:
#Initially there will be no executions since the first execution happens at the top of the hour
#Note that it is common for the execution to luanch upto 20 min after the hour.
executions = fraud_model_quality_monitor.list_executions()
executions

No executions found for schedule. monitoring_schedule_name: xgb-fraud-monitoring-schedule-2021-01-14-1531


[]

In [243]:
#Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = fraud_model_quality_monitor.describe_schedule().get("LastMonitoringExecutionSummary")
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

Waiting for first execution................................................................................................................................................................................
Execution found!


In [244]:
while not executions:
    executions = fraud_model_quality_monitor.list_executions()
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

{'ProcessingInputs': [{'InputName': 'groundtruth_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/ground_truth_data/2021-01-14-13-48-22/2021/01/14/15',
    'LocalPath': '/opt/ml/processing/groundtruth/2021/01/14/15',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/datacapture/xbg-bank-fraud-monitor-2021-01-14-13-52-13/AllTraffic/2021/01/14/15',
    'LocalPath': '/opt/ml/processing/input_data/xbg-bank-fraud-monitor-2021-01-14-13-52-13/AllTraffic/2021/01/14/15',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'r

In [245]:
status = execution['MonitoringExecutionStatus']

while status in ["Pending", "InProgress"]:
    print("Waiting for execution to finish", end="")
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job['ProcessingJobStatus'])
    print(f"{latest_job['ProcessingJobName']} job exit message, if any:", latest_job.get('ExitMessage'))
    print(f"{latest_job['ProcessingJobName']} job failure reason, if any:", latest_job.get('FailureReason'))
    sleep(30) # model quality executions consist of two Processing jobs, wait for second job to start
    latest_execution = fraud_model_quality_monitor.list_executions()[-1]
    execution = fraud_model_quality_monitor.describe_schedule()["LastMonitoringExecutionSummary"]
    status = execution['MonitoringExecutionStatus']

print("Execution status is:", status)
    
if status != 'Completed':
    print(execution)
    print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

Waiting for execution to finish...............................................................!
groundtruth-merge-202101141600-77bbff989dee09484874e8c2 job status: Completed
groundtruth-merge-202101141600-77bbff989dee09484874e8c2 job exit message, if any: None
groundtruth-merge-202101141600-77bbff989dee09484874e8c2 job failure reason, if any: None
Waiting for execution to finish...........................................................................!
model-quality-monitoring-202101141600-77bbff989dee09484874e8c2 job status: Completed
model-quality-monitoring-202101141600-77bbff989dee09484874e8c2 job exit message, if any: CompletedWithViolations: Job completed successfully with 7 violations.
model-quality-monitoring-202101141600-77bbff989dee09484874e8c2 job failure reason, if any: None
Execution status is: CompletedWithViolations
{'MonitoringScheduleName': 'xgb-fraud-monitoring-schedule-2021-01-14-1531', 'ScheduledTime': datetime.datetime(2021, 1, 14, 16, 0, tzinfo=tzlocal()), 'Creat

In [246]:
latest_execution = fraud_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
print('Report Uri:', report_uri)

Report Uri: s3://sagemaker-us-east-1-246778806733/sagemaker/fraud-ModelQualityMonitor/baselining/results/xbg-bank-fraud-monitor-2021-01-14-13-52-13/xgb-fraud-monitoring-schedule-2021-01-14-1531/2021/01/14/16


## Violations

In [248]:
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

,constraint_check_type,description,metric_name
0,LessThanThreshold,Metric precision with 0.0 +/- 0.0 was LessThanThreshold '1.0',precision
1,LessThanThreshold,Metric truePositiveRate with 0.0 +/- 0.0 was LessThanThreshold '1.0',truePositiveRate
2,LessThanThreshold,Metric f1 with 0.0 +/- 0.0 was LessThanThreshold '1.0',f1
3,LessThanThreshold,Metric accuracy with 0.6797872340425531 +/- 0.004893473713022098 was LessThanThreshold '1.0',accuracy
4,GreaterThanThreshold,Metric falseNegativeRate with 1.0 +/- 0.0 was GreaterThanThreshold '0.0',falseNegativeRate
5,LessThanThreshold,Metric recall with 0.0 +/- 0.0 was LessThanThreshold '1.0',recall
6,LessThanThreshold,Metric f2 with 0.0 +/- 0.0 was LessThanThreshold '1.0',f2


## Cloudwatch Metrics

In [249]:

# Create CloudWatch client
cw_client = boto3.Session().client('cloudwatch')

namespace='aws/sagemaker/Endpoints/model-metrics'

cw_dimensions=[
        {
            'Name': 'Endpoint',
            'Value': endpoint_name
        },
        {
            'Name': 'MonitoringSchedule',
            'Value': fraud_monitor_schedule_name
        }
]

In [251]:
# List metrics through the pagination interface
paginator = cw_client.get_paginator('list_metrics')

for response in paginator.paginate(Dimensions=cw_dimensions,Namespace=namespace):
    model_quality_metrics = response['Metrics']
    for metric in model_quality_metrics:
        print(metric['MetricName'])

f0_5_best_constant_classifier
f2_best_constant_classifier
f1_best_constant_classifier
auc
precision
accuracy_best_constant_classifier
true_positive_rate
f1
accuracy
false_positive_rate
f0_5
true_negative_rate
false_negative_rate
recall_best_constant_classifier
precision_best_constant_classifier
recall
f2


## Cloudwatch Alarm

In [252]:
alarm_name='MODEL_QUALITY_F2_SCORE'
alarm_desc='Trigger an CloudWatch alarm when the f2 score drifts away from the baseline constraints'
mdoel_quality_f2_drift_threshold=0.50 ##Setting this threshold purposefully low to see the alarm quickly.
metric_name='f2'
namespace='aws/sagemaker/Endpoints/model-metrics'

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic='Average',
    Dimensions=[
        {
            'Name': 'Endpoint',
            'Value': endpoint_name
        },
        {
            'Name': 'MonitoringSchedule',
            'Value': fraud_monitor_schedule_name
        }
    ],
    Period=600,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=mdoel_quality_f2_drift_threshold,
    ComparisonOperator='LessThanOrEqualToThreshold',
    TreatMissingData='breaching'
)

{'ResponseMetadata': {'RequestId': '67b06ba4-31f3-44f3-9dcb-2679ae892e3d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '67b06ba4-31f3-44f3-9dcb-2679ae892e3d',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Thu, 14 Jan 2021 16:23:35 GMT'},
  'RetryAttempts': 0}}

## Clean-up

In [ ]:
fraud_model_quality_monitor.delete_monitoring_schedule()

In [ ]:
predictor.delete_endpoint()
predictor.delete_model()